In [1]:
import gradio as gr
import numpy as np
import joblib

C:\Users\ENG_ABDELRAHMAN\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load model and scaler
model = joblib.load("lightgbm_model.pkl")
scaler = joblib.load("scaler.pkl")


In [3]:
# Define prediction function
def predict_heart_failure(age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction,
                          high_blood_pressure, platelets, serum_creatinine, serum_sodium,
                          sex, smoking, time):

    # Feature engineering
    platelets_per_age = platelets / (age + 1)
    creatinine_per_ck = serum_creatinine / (creatinine_phosphokinase + 1)
    ejection_per_age = ejection_fraction / (age + 1)
    anaemia_creatinine = anaemia * serum_creatinine

    # Create input array
    features = np.array([[age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction,
                          high_blood_pressure, platelets, serum_creatinine, serum_sodium,
                          sex, smoking, time,
                          platelets_per_age, creatinine_per_ck, ejection_per_age, anaemia_creatinine]])

    # Scale input
    features_scaled = scaler.transform(features)

    # Predict
    prediction = model.predict(features_scaled)[0]
    return "✅ Survived" if prediction == 0 else "❌ Death Expected"

In [4]:
# Gradio interface
inputs = [
    gr.Number(label="Age"),
    gr.Radio([0, 1], label="Anaemia (0=No, 1=Yes)"),
    gr.Number(label="Creatinine Phosphokinase"),
    gr.Radio([0, 1], label="Diabetes (0=No, 1=Yes)"),
    gr.Number(label="Ejection Fraction"),
    gr.Radio([0, 1], label="High Blood Pressure"),
    gr.Number(label="Platelets"),
    gr.Number(label="Serum Creatinine"),
    gr.Number(label="Serum Sodium"),
    gr.Radio([0, 1], label="Sex (0=Female, 1=Male)"),
    gr.Radio([0, 1], label="Smoking (0=No, 1=Yes)"),
    gr.Number(label="Follow-up Time"),
]

app = gr.Interface(fn=predict_heart_failure, inputs=inputs, outputs="text",
                   title="🫀 Heart Failure Predictor",
                   description="Predicts if a heart failure patient is likely to die (DEATH_EVENT) based on medical data.")

In [5]:
app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
